## Extract, Transform and Load (ETL) Step

We are going to use the YOLO algorithm for object detection. Let's transform the dataset into the format expected by YOLO

https://docs.ultralytics.com/datasets/detect/#ultralytics-yolo-format

### Setup

In [1]:
import os
import random
import time
from dataclasses import dataclass
from pathlib import Path
from enum import Enum
import shutil


ROOT_DIR = "datasets\\aircraft"
LABELS_DIR = "labels"
IMAGES_DIR = "images"
TRAIN_DIR_LABELS = os.path.join(ROOT_DIR, LABELS_DIR, "train")
VAL_DIR_LABELS = os.path.join(ROOT_DIR, LABELS_DIR, "val")
TEST_DIR_LABELS = os.path.join(ROOT_DIR, LABELS_DIR, "test")
TRAIN_DIR_IMAGES = os.path.join(ROOT_DIR, IMAGES_DIR, "train")
VAL_DIR_IMAGES = os.path.join(ROOT_DIR, IMAGES_DIR, "val")
TEST_DIR_IMAGES = os.path.join(ROOT_DIR, IMAGES_DIR, "test")
DATASET_DIR = "..\\dataset\\dataset"

## Classes Mapping

We also need to provide a classes mapping. We only have one class since we're only trying to detect airplanes, not identify them.

In [2]:
CLASSES_RAW = [
    "aircraft"
]

CLASSES = {
    "aircraft": 0
}

print(CLASSES)

{'aircraft': 0}


### Transforming and Normalizing Bounding Boxes

Since the dataset CSV is given using this format
`filename, width, height, class, xmin, ymin, xmax, ymax`

We need to turn it into a **normalized xywh format**, with the center point, bounding box width and height, all normalized to [0; 1]

The normalize_bounding_box function does just that

In [3]:
def normalize_bounding_box(
    x_min: int, y_min: int, x_max: int, y_max: int, width: int, height: int
) -> tuple[float, float, float, float]:
    """Generates normalized YOLO format xywh coordinates from bounding box pixel coordinates.

    Args:
        x_min (int): min x pixel from bounding box
        y_min (int): min y pixel from bounding box
        x_max (int): max x pixel from bounding box
        y_max (int): max y pixel from bounding box
        width (int): width of the image in pixels
        height (int): height of the image in pixels

    Returns:
        x_center (float): x coordinates of the center of the bounding box [0; 1]
        y_center (float): y coordinates of the center of the bounding box [0; 1]
        width (float): width of the bounding box
        height (float): height of the bounding box
    """

    # First, we transform the min-max bounding box to center
    center_x = (x_min + x_max) / 2
    center_y = (y_min + y_max) / 2

    bb_width = x_max - x_min
    bb_height = y_max - y_min

    return (center_x / width, center_y / height, bb_width / width, bb_height / height)

# Let's test it with a line in our dataset
normalize_bounding_box(852, 177, 1998, 503, 2048, 1365)

(0.69580078125, 0.2490842490842491, 0.5595703125, 0.23882783882783884)

### Extracting and Transforming the Dataset

We need to turn our dataset in the YOLO format, using a very specific directory structure.

### Splitting the Dataset

We need to split out dataset into three parts, train, validation and test.

We're going to use (80%, 10%, 10%) repartition for now.

In [7]:
AIRPLANE_CLASS_STR = "aircraft"

class EntryType(Enum):
    TRAIN = "train"
    VAL = "val"
    TEST = "test"

    

def split_dataset(seed: int = None) -> EntryType:
    if not seed:
        seed = time.time()

    random.seed(seed)

    rand = random.randint(1, 10)

    type = None

    # Validation (10%)
    if rand == 9:
        type = EntryType.VAL

    # Test (10%)
    elif rand == 10:
        type = EntryType.TEST

    # Train (80%)
    else:
        type = EntryType.TRAIN

    return type


@dataclass
class CSVEntry:
    filename: str
    img_width: int
    img_height: int
    airplane_class: str
    x_min: int
    y_min: int
    x_max: int
    y_max: int


def process_csv_file(filename: str) -> list[CSVEntry]:
    with open(filename, "r") as f:
        lines = f.readlines()

    result = []

    # Line 0 is for the column names
    for line in lines[1:]:
        entries = line.strip().split(",")
        filename = entries[0]
        img_width = int(entries[1])
        img_height = int(entries[2])
        airplane_class = AIRPLANE_CLASS_STR # Hardcode airplane class for detection only
        x_min = int(entries[4])
        y_min = int(entries[5])
        x_max = int(entries[6])
        y_max = int(entries[7])

        result.append(
        CSVEntry(
            filename, img_width, img_height, airplane_class, x_min, y_min, x_max, y_max
        )
    )

    return result

def create_yolo_txt(entries: list[CSVEntry], type: str, filename: str) -> bool:
    to_dir = ""

    if type == EntryType.TRAIN:
        to_dir = TRAIN_DIR_LABELS
    
    elif type == EntryType.VAL:
        to_dir = VAL_DIR_LABELS

    else:
        to_dir = TEST_DIR_LABELS
        

    os.makedirs(to_dir, exist_ok=True)

    fname_noext = Path(filename).stem

    to_create = os.path.join(to_dir, fname_noext + ".txt")


    with open(to_create, "w") as f:
        for entry in entries:
            classid = CLASSES[entry.airplane_class.lower()]
            x_center, y_center, width, height = normalize_bounding_box(entry.x_min, entry.y_min, entry.x_max, entry.y_max, entry.img_width, entry.img_height)
            f.write(' '.join(map(str, [classid, x_center, y_center, width, height])) + '\n')
    
    print(f"Wrote {to_create}")

    return True

def copy_image(type: str, filename: str) -> bool:

    to_dir = ""

    if type == EntryType.TRAIN:
        to_dir = TRAIN_DIR_IMAGES
    
    elif type == EntryType.VAL:
        to_dir = VAL_DIR_IMAGES

    else:
        to_dir = TEST_DIR_IMAGES

    os.makedirs(to_dir, exist_ok=True)

    to_copy = os.path.join(DATASET_DIR, filename + ".jpg")

    to_paste = os.path.join(to_dir, filename + ".jpg")

    shutil.copyfile(to_copy, to_paste)

    return True


def extract_transform_yolo(dataset_dir: str = DATASET_DIR, seed: int = None) -> None:
    directory = os.fsencode(dataset_dir)

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        # Only process CSV files
        if not filename.endswith("csv"):
            continue
        
        to_open = os.path.join(dataset_dir, filename)
        entries = process_csv_file(to_open)

        to_check = os.path.join(dataset_dir, entries[0].filename + ".jpg")
        if not os.path.exists(to_check):
            print("File associated with CSV not found!")
            continue

        print(f"Processing: {filename}")


        # Determine where to add the entry (train, validation, test)
        type = split_dataset()

        if not create_yolo_txt(entries, type, filename):
            print(f"Error creating {filename} to {type}!")

        if not copy_image(type, entries[0].filename):
             print(f"Error copying {filename} to {type}!")

        
        # time.sleep(1)

    print("ETL done!")

extract_transform_yolo()

Processing: 0000e97ea2d086d6759b19b288a8a72c.csv
Wrote datasets\aircraft\labels\train\0000e97ea2d086d6759b19b288a8a72c.txt
Processing: 00010041af654d0b8e1e16c824fa9867.csv
Wrote datasets\aircraft\labels\train\00010041af654d0b8e1e16c824fa9867.txt
Processing: 000106393cfe2343888c584e65fd2274.csv
Wrote datasets\aircraft\labels\train\000106393cfe2343888c584e65fd2274.txt
Processing: 0002f2cc10d90392efc3ad30cbda6490.csv
Wrote datasets\aircraft\labels\train\0002f2cc10d90392efc3ad30cbda6490.txt
Processing: 00032844ab679240fc03ecd27d29a6aa.csv
Wrote datasets\aircraft\labels\train\00032844ab679240fc03ecd27d29a6aa.txt
Processing: 0003f56298fa8999168d7988a2e9549d.csv
Wrote datasets\aircraft\labels\val\0003f56298fa8999168d7988a2e9549d.txt
Processing: 000aa01b25574f28b654718db0700f72.csv
Wrote datasets\aircraft\labels\train\000aa01b25574f28b654718db0700f72.txt
Processing: 000b7e09368a2500506063d27d90fcc0.csv
Wrote datasets\aircraft\labels\train\000b7e09368a2500506063d27d90fcc0.txt
Processing: 000bdd